In [0]:
!pip install tensorflow==2.0.0-alpha0
!pip install tfp-nightly
#!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function
from platform import python_version
 
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras

print(tf.__version__)
print(python_version())
print(pd.__version__)

In [0]:
column_names = ['Date','HomeTeam','HomeScore','AwayTeam','AwayScore']
team_columns = ['Conference','Team']

In [0]:
games_csv = 'https://raw.githubusercontent.com/ehennis/BasketballScout/master/Data/Games.csv'
games = pd.read_csv(games_csv, header=None, names=column_names)

In [0]:
games = games.drop( games[(games['HomeScore'] == 0)].index )

## Win/Loss Record

In [0]:
games.loc[ (games.HomeScore > games.AwayScore),'HomeResult'] = 1
games.loc[ (games.HomeScore < games.AwayScore),'HomeResult'] = 0

games.loc[ (games.HomeScore < games.AwayScore),'AwayResult'] = 1
games.loc[ (games.HomeScore > games.AwayScore),'AwayResult'] = 0

## Break into Seasons

In [0]:
s2015 = games[(games['Date'] > '2015-11-01') & (games['Date'] < '2016-04-15')].copy()
s2016 = games[(games['Date'] > '2016-11-01') & (games['Date'] < '2017-04-15')].copy()
s2017 = games[(games['Date'] > '2017-11-01') & (games['Date'] < '2018-04-15')].copy()
s2018 = games[(games['Date'] > '2018-11-01') & (games['Date'] < '2019-04-15')].copy()

print('2015 (%s): %s - %s' % (s2015.shape[0],np.min(s2015.Date),np.max(s2015.Date)))
print('2016 (%s): %s - %s' % (s2016.shape[0],np.min(s2016.Date),np.max(s2016.Date)))
print('2017 (%s): %s - %s' % (s2017.shape[0],np.min(s2017.Date),np.max(s2017.Date)))
print('2018 (%s): %s - %s' % (s2018.shape[0],np.min(s2018.Date),np.max(s2018.Date)))

## Load/Clean Teams

In [0]:
teams_csv = 'https://raw.githubusercontent.com/ehennis/BasketballScout/master/Data/Teams.csv'
teams = pd.read_csv(teams_csv, header=None, names=team_columns)

**Add Conference  Code**
I am going to add the conference in the hopes that it can determine pre conference and major mismatches

In [0]:
#Add a category code to the Teams data frame
teams['ConfCode'] = pd.Categorical(teams.Conference).codes
teams.head()

**Alter Team Names**  
The first 3 seasons I scraped used shortened names. 2018 does not. Instead of changing all of those I am just going to change the values in 2018-2019 season

In [0]:
def RenameTeams(df_games, column_name):
  df_games.loc[ df_games[column_name] == 'A&M-Corpus Chris'		, column_name ] = 		'Texas A&M-CC'	
  df_games.loc[ df_games[column_name] == 'Alabama St.'		, column_name ] = 		'Alabama State'		
  df_games.loc[ df_games[column_name] == 'Albany (NY)'		, column_name ] = 		'Albany'				
  df_games.loc[ df_games[column_name] == 'Alcorn St.'			, column_name ] = 		'Alcorn State'		
  df_games.loc[ df_games[column_name] == 'American'			, column_name ] = 		'American University'
  df_games.loc[ df_games[column_name] == 'Appalachian St.'			, column_name ] = 		'Appalachian State'	
  df_games.loc[ df_games[column_name] == 'Arizona St.'		, column_name ] = 		'Arizona State'						
  df_games.loc[ df_games[column_name] == 'Army West Point'		, column_name ] = 		'Army'					
  df_games.loc[ df_games[column_name] == 'Ark.-Pine Bluff'		, column_name ] = 		'Arkansas-Pine Bluff'
  df_games.loc[ df_games[column_name] == 'UALR'				, column_name ] = 		'Arkansas-Little Rock'	
  df_games.loc[ df_games[column_name] == 'Little Rock'				, column_name ] = 		'Arkansas-Little Rock'			
  df_games.loc[ df_games[column_name] == 'Arkansas St.'		, column_name ] = 		'Arkansas State'		
  df_games.loc[ df_games[column_name] == 'Ball St.'			, column_name ] = 		'Ball State'			
  df_games.loc[ df_games[column_name] == 'Boise St.'			, column_name ] = 		'Boise State'		
  df_games.loc[ df_games[column_name] == 'Boston U.'			, column_name ] = 		'Boston University'			
  df_games.loc[ df_games[column_name] == 'Cal Baptist'	, column_name ] = 		'California Baptist'			
  df_games.loc[ df_games[column_name] == 'Charleston So.'	, column_name ] = 		'Charleston Southern'			
  df_games.loc[ df_games[column_name] == 'Cent. Conn. St.'	, column_name ] = 		'Central Connecticut State'	
  df_games.loc[ df_games[column_name] == 'Central Conn. St.'	, column_name ] = 		'Central Connecticut State'	
  df_games.loc[ df_games[column_name] == 'Central Mich.'	, column_name ] = 		'Central Michigan'	
  df_games.loc[ df_games[column_name] == 'Col. of Charleston'	, column_name ] = 		'Charleston'			
  df_games.loc[ df_games[column_name] == 'Chicago St.'		, column_name ] = 		'Chicago State'		
  df_games.loc[ df_games[column_name] == 'Cleveland St.'		, column_name ] = 		'Cleveland State'		
  df_games.loc[ df_games[column_name] == 'Coastal Caro.'		, column_name ] = 		'Coastal Carolina'				
  df_games.loc[ df_games[column_name] == 'Colorado St.'		, column_name ] = 		'Colorado State'	
  df_games.loc[ df_games[column_name] == 'Coppin St.'			, column_name ] = 		'Coppin State'			
  df_games.loc[ df_games[column_name] == 'Bakersfield'		, column_name ] = 		'Cal State Bakersfield'	
  df_games.loc[ df_games[column_name] == 'CSU Bakersfield'		, column_name ] = 		'Cal State Bakersfield'		
  df_games.loc[ df_games[column_name] == 'Bryant'		, column_name ] = 		'Bryant University'	
  df_games.loc[ df_games[column_name] == 'Cal St. Fullerton'	, column_name ] = 		'Cal State Fullerton'
  df_games.loc[ df_games[column_name] == 'CSU Fullerton'	, column_name ] = 		'Cal State Fullerton'		
  df_games.loc[ df_games[column_name] == 'CSUN'	, column_name ] = 		'Cal State Northridge'	
  df_games.loc[ df_games[column_name] == 'Cal St. Northridge'	, column_name ] = 		'Cal State Northridge'						
  df_games.loc[ df_games[column_name] == 'Central Ark.'		, column_name ] = 		'Central Arkansas'						
  df_games.loc[ df_games[column_name] == 'Delaware St.'		, column_name ] = 		'Delaware State'		
  df_games.loc[ df_games[column_name] == 'Detroit'			, column_name ] = 		'Detroit Mercy'		
  df_games.loc[ df_games[column_name] == 'East Tenn. St.'		, column_name ] = 		'East Tennessee State'
  df_games.loc[ df_games[column_name] == 'Eastern Ill.'		, column_name ] = 		'Eastern Illinois'		
  df_games.loc[ df_games[column_name] == 'Eastern Ky.'		, column_name ] = 		'Eastern Kentucky'		
  df_games.loc[ df_games[column_name] == 'Eastern Mich.'		, column_name ] = 		'Eastern Michigan'	
  df_games.loc[ df_games[column_name] == 'Eastern Wash.'		, column_name ] = 		'Eastern Washington'
  df_games.loc[ df_games[column_name] == "Fairleigh D'son"		, column_name ] = 		'Fairleigh Dickinson'				
  df_games.loc[ df_games[column_name] == 'FGCU'		, column_name ] = 		'Florida Gulf Coast'						
  df_games.loc[ df_games[column_name] == 'FIU'				, column_name ] = 		'Florida International'					
  df_games.loc[ df_games[column_name] == 'Fla. Atlantic'		, column_name ] = 		'Florida Atlantic'
  df_games.loc[ df_games[column_name] == 'Florida St.'		, column_name ] = 		'Florida State'			
  df_games.loc[ df_games[column_name] == 'Fresno St.'			, column_name ] = 		'Fresno State'		
  df_games.loc[ df_games[column_name] == 'Fort Wayne'		, column_name ] = 		'Purdue Fort Wayne'		
  df_games.loc[ df_games[column_name] == 'IPFW'				, column_name ] = 		'Purdue Fort Wayne'				
  df_games.loc[ df_games[column_name] == 'Ga. Southern'		, column_name ] = 		'Georgia Southern'			
  df_games.loc[ df_games[column_name] == 'Georgia St.'		, column_name ] = 		'Georgia State'			
  df_games.loc[ df_games[column_name] == 'Geo. Washington'		, column_name ] = 		'George Washington'				
  df_games.loc[ df_games[column_name] == 'Grambling'		, column_name ] = 		'Grambling State'		
  df_games.loc[ df_games[column_name] == 'Humboldt St.'		, column_name ] = 		'Humboldt State'		
  df_games.loc[ df_games[column_name] == 'Idaho St.'			, column_name ] = 		'Idaho State'			
  df_games.loc[ df_games[column_name] == 'Illinois St.'		, column_name ] = 		'Illinois State'		
  df_games.loc[ df_games[column_name] == 'Iowa St.'			, column_name ] = 		'Iowa State'			
  df_games.loc[ df_games[column_name] == 'Indiana St.'		, column_name ] = 		'Indiana State'		
  df_games.loc[ df_games[column_name] == 'Jackson St.'		, column_name ] = 		'Jackson State'		
  df_games.loc[ df_games[column_name] == 'Jacksonville St.'		, column_name ] = 		'Jacksonville State'			
  df_games.loc[ df_games[column_name] == 'Kansas St.'		, column_name ] = 		'Kansas State'
  df_games.loc[ df_games[column_name] == 'Kennesaw St.'		, column_name ] = 		'Kennesaw State'		
  df_games.loc[ df_games[column_name] == 'Kent St.'			, column_name ] = 		'Kent State'			
  df_games.loc[ df_games[column_name] == 'Louisiana'		, column_name ] = 		'Louisiana-Lafayette'
  df_games.loc[ df_games[column_name] == 'Lamar University'		, column_name ] = 		'Lamar'	
  df_games.loc[ df_games[column_name] == 'La.-Monroe'		, column_name ] = 		'Louisiana-Monroe'		
  df_games.loc[ df_games[column_name] == 'Long Beach St.'		, column_name ] = 		'Long Beach State'	
  df_games.loc[ df_games[column_name] == 'Long Island'		, column_name ] = 		'LIU Brooklyn'
  df_games.loc[ df_games[column_name] == 'LMU'	, column_name ] = 		'Loyola Marymount'					
  df_games.loc[ df_games[column_name] == 'Loyola Chicago'	, column_name ] = 		'Loyola (IL)'			
  df_games.loc[ df_games[column_name] == 'Loyola Maryland'	, column_name ] = 		'Loyola (MD)'			
  df_games.loc[ df_games[column_name] == 'Loyola (Md.)'	, column_name ] = 		'Loyola (MD)'		
  df_games.loc[ df_games[column_name] == 'UMES'		, column_name ] = 		'Maryland-Eastern Shore'
  df_games.loc[ df_games[column_name] == 'Miami (Fla.)'		, column_name ] = 		'Miami (FL)'
  df_games.loc[ df_games[column_name] == 'Miami (Ohio)'		, column_name ] = 		'Miami (OH)'
  df_games.loc[ df_games[column_name] == "Mt. St. Mary's"		, column_name ] = 		"Mount St Mary's"			
  df_games.loc[ df_games[column_name] == 'Mass.-Lowell'		, column_name ] = 		'Massachusetts-Lowell'				
  df_games.loc[ df_games[column_name] == 'McNeese'		, column_name ] = 		'McNeese State'										
  df_games.loc[ df_games[column_name] == 'McNeese '		, column_name ] = 		'McNeese State'						
  df_games.loc[ df_games[column_name] == 'McNeese St.'		, column_name ] = 		'McNeese State'			
  df_games.loc[ df_games[column_name] == 'Middle Tenn.'		, column_name ] = 		'Middle Tennessee'		
  df_games.loc[ df_games[column_name] == 'Mississippi St.'		, column_name ] = 		'Mississippi State'	
  df_games.loc[ df_games[column_name] == 'Mississippi Val.'		, column_name ] = 		'Mississippi Valley State'			
  df_games.loc[ df_games[column_name] == 'Mich. St. '		, column_name ] = 		'Michigan State'		
  df_games.loc[ df_games[column_name] == 'Michigan St.'		, column_name ] = 		'Michigan State'			
  df_games.loc[ df_games[column_name] == 'Mississippi'			, column_name ] = 		'Ole Miss'			
  df_games.loc[ df_games[column_name] == 'Missouri St.'		, column_name ] = 		'Missouri State'	
  df_games.loc[ df_games[column_name] == 'Montana St.'		, column_name ] = 		'Montana State'		
  df_games.loc[ df_games[column_name] == 'Morehead St.'		, column_name ] = 		'Morehead State'		
  df_games.loc[ df_games[column_name] == 'Morgan St.'			, column_name ] = 		'Morgan State'			
  df_games.loc[ df_games[column_name] == 'Murray St.'			, column_name ] = 		'Murray State'		
  df_games.loc[ df_games[column_name] == 'N.C. A&T'	, column_name ] = 		'North Carolina A&T'
  df_games.loc[ df_games[column_name] == 'N.C. Central'	, column_name ] = 		'North Carolina Central'
  df_games.loc[ df_games[column_name] == 'New Mexico St.'	, column_name ] = 		'New Mexico State'				
  df_games.loc[ df_games[column_name] == 'NC State'	, column_name ] = 		'North Carolina State'
  df_games.loc[ df_games[column_name] == 'North Carolina St.'	, column_name ] = 		'North Carolina State'	
  df_games.loc[ df_games[column_name] == 'North Dakota St.'	, column_name ] = 		'North Dakota State'				
  df_games.loc[ df_games[column_name] == 'Northern Ariz.'	, column_name ] = 		'Northern Arizona'				
  df_games.loc[ df_games[column_name] == 'Northern Colo.'	, column_name ] = 		'Northern Colorado'		
  df_games.loc[ df_games[column_name] == 'Northern Ill.'	, column_name ] = 		'Northern Illinois'			
  df_games.loc[ df_games[column_name] == "N'western St."	, column_name ] = 		"Northwestern State"	
  df_games.loc[ df_games[column_name] == 'Northwestern St.'	, column_name ] = 		"Northwestern State"				
  df_games.loc[ df_games[column_name] == 'Nicholls St.'		, column_name ] = 		'Nicholls State'		
  df_games.loc[ df_games[column_name] == 'Norfolk St.'		, column_name ] = 		'Norfolk State'		
  df_games.loc[ df_games[column_name] == 'Northern Ky.'		, column_name ] = 		'Northern Kentucky'	
  df_games.loc[ df_games[column_name] == 'Ohio St.'			, column_name ] = 		'Ohio State'			
  df_games.loc[ df_games[column_name] == 'Ohio St. '			, column_name ] = 		'Ohio State'			
  df_games.loc[ df_games[column_name] == 'Oklahoma St.'		, column_name ] = 		'Oklahoma State'	
  df_games.loc[ df_games[column_name] == 'Oregon St.'		, column_name ] = 		'Oregon State'	
  df_games.loc[ df_games[column_name] == 'Neb. Omaha'			, column_name ] = 		'Nebraska-Omaha'	
  df_games.loc[ df_games[column_name] == 'Omaha'			, column_name ] = 		'Nebraska-Omaha'					
  df_games.loc[ df_games[column_name] == 'Penn'			, column_name ] = 		'Pennsylvania'								
  df_games.loc[ df_games[column_name] == 'Penn St.'			, column_name ] = 		'Penn State'				
  df_games.loc[ df_games[column_name] == 'Prairie View'		, column_name ] = 		'Prairie View A&M'		
  df_games.loc[ df_games[column_name] == 'Portland St.'		, column_name ] = 		'Portland State'		
  df_games.loc[ df_games[column_name] == 'S.C. Upstate'	, column_name ] = 		'USC Upstate'	
  df_games.loc[ df_games[column_name] == 'S. Carolina St.'	, column_name ] = 		'South Carolina State'
  df_games.loc[ df_games[column_name] == 'South Carolina St.'	, column_name ] = 		'South Carolina State'			
  df_games.loc[ df_games[column_name] == 'Sacramento St.'		, column_name ] = 		'Sacramento State'			
  df_games.loc[ df_games[column_name] == 'Sam Houston St.'		, column_name ] = 		'Sam Houston State'				
  df_games.loc[ df_games[column_name] == 'San Diego St.'		, column_name ] = 		'San Diego State'		
  df_games.loc[ df_games[column_name] == 'San Jose St.'		, column_name ] = 		'San Jose State'		
  df_games.loc[ df_games[column_name] == 'Savannah St.'		, column_name ] = 		'Savannah State'		
  df_games.loc[ df_games[column_name] == 'Seattle U'			, column_name ] = 		'Seattle'			
  df_games.loc[ df_games[column_name] == 'SFA'	, column_name ] = 		'Stephen F Austin'
  df_games.loc[ df_games[column_name] == 'Stephen F. Austin'	, column_name ] = 		'Stephen F Austin'					
  df_games.loc[ df_games[column_name] == 'SIU Edwardsville'	, column_name ] = 		'SIU-Edwardsville'						
  df_games.loc[ df_games[column_name] == 'SIUE'	, column_name ] = 		'SIU-Edwardsville'					
  df_games.loc[ df_games[column_name] == 'South Ala.'			, column_name ] = 		'South Alabama'			
  df_games.loc[ df_games[column_name] == 'South Dakota St.'			, column_name ] = 		'South Dakota State'	
  df_games.loc[ df_games[column_name] == 'South Fla.'			, column_name ] = 		'South Florida'		
  df_games.loc[ df_games[column_name] == 'Southern Ill.'		, column_name ] = 		'Southern Illinois'	
  df_games.loc[ df_games[column_name] == 'Southeast Mo. St.'		, column_name ] = 		'Southeast Missouri State'
  df_games.loc[ df_games[column_name] == 'Southeastern La.'		, column_name ] = 		'Southeastern Louisiana'		
  df_games.loc[ df_games[column_name] == 'Southern Miss.'		, column_name ] = 		'Southern Miss'		
  df_games.loc[ df_games[column_name] == 'Southern U.'		, column_name ] = 		'Southern University'	
  df_games.loc[ df_games[column_name] == 'Southern Univ.'		, column_name ] = 		'Southern University'	
  df_games.loc[ df_games[column_name] == "St. Bonaventure"	, column_name ] = 		"St Bonaventure"			
  df_games.loc[ df_games[column_name] == "St. Francis (B'klyn)"	, column_name ] = 		"St Francis (BKN)"			
  df_games.loc[ df_games[column_name] == 'St. Francis (NY)'	, column_name ] = 		"St Francis (BKN)"
  df_games.loc[ df_games[column_name] == 'St. Francis (PA)'	, column_name ] = 		"St Francis (PA)"	
  df_games.loc[ df_games[column_name] == 'St. Francis (Pa.)'	, column_name ] = 		"St Francis (PA)"	
  df_games.loc[ df_games[column_name] == "Saint Joseph's"	, column_name ] = 		"Saint Joseph's (PA)"
  df_games.loc[ df_games[column_name] == "St. Mary's (CA)"	, column_name ] = 		"Saint Mary's"	
  df_games.loc[ df_games[column_name] == "St. Mary's (Cal.)"	, column_name ] = 		"Saint Mary's"	
  df_games.loc[ df_games[column_name] == "St. Peter's"	, column_name ] = 		"St Peter's"	
  df_games.loc[ df_games[column_name] == "St. John's (NY)"	, column_name ] = 		"St John's"		
  df_games.loc[ df_games[column_name] == "St. John's "	, column_name ] = 		"St John's"				
  df_games.loc[ df_games[column_name] == 'Tennessee St.'		, column_name ] = 		'Tennessee State'		
  df_games.loc[ df_games[column_name] == 'Texas A&M-C.C.'			, column_name ] = 		'Texas A&M-CC'			
  df_games.loc[ df_games[column_name] == 'Texas St.'			, column_name ] = 		'Texas State'			
  df_games.loc[ df_games[column_name] == 'UC Santa Barbara'	, column_name ] = 		'UC Santa Barb.'		
  df_games.loc[ df_games[column_name] == 'Ill.-Chicago'		, column_name ] = 		'UIC'					
  df_games.loc[ df_games[column_name] == 'Md.-East. Shore'	, column_name ] = 		'Maryland-Eastern Shore'					
  df_games.loc[ df_games[column_name] == 'UNCG'		, column_name ] = 		'UNC Greensboro'					
  df_games.loc[ df_games[column_name] == 'UNCW'		, column_name ] = 		'North Carolina-Wilmington'					
  df_games.loc[ df_games[column_name] == 'UNC Wilmington'		, column_name ] = 		'North Carolina-Wilmington'					
  df_games.loc[ df_games[column_name] == 'Southern California', column_name ] = 		'USC'					
  df_games.loc[ df_games[column_name] == 'UConn'			, column_name ] = 		'Connecticut'					
  df_games.loc[ df_games[column_name] == 'UC Santa Barb.'			, column_name ] = 		'UC Santa Barbara'						
  df_games.loc[ df_games[column_name] == 'UIC'			, column_name ] = 		'Illinois-Chicago'						
  df_games.loc[ df_games[column_name] == 'UNI'			, column_name ] = 		'Northern Iowa'							
  df_games.loc[ df_games[column_name] == 'UT Arlington'			, column_name ] = 		'Texas-Arlington'		
  df_games.loc[ df_games[column_name] == 'UT Arlington '			, column_name ] = 		'Texas-Arlington'		
  df_games.loc[ df_games[column_name] == 'UT Martin'			, column_name ] = 		'Tennessee-Martin'		
  df_games.loc[ df_games[column_name] == 'UTRGV'			, column_name ] = 		'Texas Rio Grande Valley'		
  df_games.loc[ df_games[column_name] == 'Utah St.'			, column_name ] = 		'Utah State'				
  df_games.loc[ df_games[column_name] == 'VCU'			, column_name ] = 		'Virginia Commonwealth'		
  df_games.loc[ df_games[column_name] == 'VMI'			, column_name ] = 		'Virginia Military'		
  df_games.loc[ df_games[column_name] == 'Washington St.'			, column_name ] = 		'Washington State'			
  df_games.loc[ df_games[column_name] == 'Weber St.'			, column_name ] = 		'Weber State'			
  df_games.loc[ df_games[column_name] == 'Western Caro.'		, column_name ] = 		'Western Carolina'
  df_games.loc[ df_games[column_name] == 'Western Ill.'		, column_name ] = 		'Western Illinois'	
  df_games.loc[ df_games[column_name] == 'Western Ky.'		, column_name ] = 		'Western Kentucky'	
  df_games.loc[ df_games[column_name] == 'Western Mich.'		, column_name ] = 		'Western Michigan'		
  df_games.loc[ df_games[column_name] == 'Wichita St.'		, column_name ] = 		'Wichita State'		
  df_games.loc[ df_games[column_name] == 'Wright St.'			, column_name ] = 		'Wright State'	
  df_games.loc[ df_games[column_name] == 'Youngstown St.'			, column_name ] = 		'Youngstown State'	

In [0]:
RenameTeams(s2015,'HomeTeam')
RenameTeams(s2015,'AwayTeam')
RenameTeams(s2016,'HomeTeam')
RenameTeams(s2016,'AwayTeam')
RenameTeams(s2017,'HomeTeam')
RenameTeams(s2017,'AwayTeam')
RenameTeams(s2018,'HomeTeam')
RenameTeams(s2018,'AwayTeam')


## Find Points Per Game on Offense and Defense

* Break into teams
* Determine Home PPG and Away PPG
* Determine Home Points Against and Away Points Against

In [0]:
def CalculateScores(df):
  for index, row in teams.iterrows():
    ## Home Team Stats
    df.loc[ df['HomeTeam'] == row.Team, 'HomeScoreAverage' ] = df[(df['HomeTeam'] == row.Team)]['HomeScore'].expanding().mean()
    df.loc[ df['HomeTeam'] == row.Team, 'HomeDefenseAverage' ] = df[(df['HomeTeam'] == row.Team)]['AwayScore'].expanding().mean()
    ## Away Team Stats
    df.loc[ df['AwayTeam'] == row.Team, 'AwayScoreAverage' ] = df[(df['AwayTeam'] == row.Team)]['AwayScore'].expanding().mean()
    df.loc[ df['AwayTeam'] == row.Team, 'AwayDefenseAverage' ] = df[(df['AwayTeam'] == row.Team)]['HomeScore'].expanding().mean()
    


CalculateScores(s2015)
CalculateScores(s2016)
CalculateScores(s2017)
CalculateScores(s2018)

## Combine the data sets

In [0]:
all_data = pd.concat([s2015,s2016,s2017,s2018])


**Conference Code**  
Add the conference code

In [0]:
all_data = all_data.merge(teams, how='left', left_on='HomeTeam', right_on='Team')
all_data = all_data.merge(teams, how='left', left_on='AwayTeam', right_on='Team')
all_data.rename(columns={'ConfCode_x' : 'HomeConf'}, inplace=True)
all_data.rename(columns={'ConfCode_y' : 'AwayConf'}, inplace=True)
all_data.drop(['Team_x','Team_y','Conference_x','Conference_y'], axis=1, inplace=True)
all_data.head()

**Clean Data**  
We have some D2 and Juco teams that won't have a scoring average so we need to drop those rows.

In [0]:
all_data = all_data.dropna()

## Add Label

In [0]:
all_data['Result'] = games.HomeScore - games.AwayScore
all_data.tail()

## Machine Learn All The Things

**TensorBoard**  
Setting up the TensorBoard values

In [0]:
import datetime, os
%load_ext tensorboard.notebook
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

**PrintDoc**  
Create a method to supress the typical output to make it "cleaner" looking

In [0]:
class PrintDoc(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

I don't need HomeTeam, AwayTeam, HomeResult, AwayResult so I can remove them.

In [0]:
all_data = all_data.drop(['Date','HomeTeam','HomeScore','AwayTeam','AwayScore','HomeResult', 'AwayResult'], axis=1)
all_data.tail()

In [0]:
#Break it into 80/20 splits
train = all_data.sample(frac=0.8, random_state=0)
test = all_data.drop(train.index)
print('Training Size: %s' % train.shape[0])
print('Testing Size: %s' % test.shape[0])

#Create the labels
train_labels = train.pop('Result')
test_labels = test.pop('Result')

In [0]:
mean = train.mean(axis=0)
train_data = train - mean
std = train_data.std(axis=0)
train_data /= std

test_data = test - mean
test_data /= std

**Training the model**

Creating a method to build the model

In [0]:
def Build_Model():
  model = keras.models.Sequential([
      keras.layers.Dense(32, activation='relu', input_shape=[len(train.keys())]),
      keras.layers.Dense(32, activation='relu'),
      keras.layers.Dense(1) #No last layer activation for continous prediction
  ])
  
  optimizer = keras.optimizers.RMSprop()
  
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae','accuracy', 'mse'])
  return model

In [0]:
train.drop(['HomeConf','AwayConf'], axis=1, inplace=True)
train_data.drop(['HomeConf','AwayConf'], axis=1, inplace=True)
train_data.head()

In [0]:
model = Build_Model()
#Train
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2, verbose=0, callbacks=[PrintDoc(),tensorboard_callback])

In [0]:
%tensorboard --logdir logs